<a href="https://colab.research.google.com/github/fermasia/infovis/blob/gh-pages/Armado_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
from datetime import datetime, timedelta
from google.colab import files

In [ ]:
# Importación Dataset CASOS
# Variables que nos interesan:  fecha, 
#                               residencia_provincia_id, 
#                               id_departamento, 
#                               cantidad contagios (fecha_diagnostico), 
#                               cantidad internaciones (fecha_internacion), 
#                               cantidad cuidado intensivo (fecha_cui_intensivo), 
#                               cantidad muertes (fecha_fallecimiento)
start_time = time.time()
casos = None
while casos is None:
  try:
    casos = pd.read_csv('https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.zip')
  except:
    pass
print("--- %s seconds ---" % (time.time() - start_time))

--- 103.55275630950928 seconds ---


In [ ]:
# Quedarnos con las columnas que necesitamos y borrar filas con NaNs
casos = casos[((casos.clasificacion_resumen == 'Confirmado') | (casos.clasificacion_resumen == 'Sospechoso'))]
casos = casos[['id_evento_caso','residencia_provincia_id','fecha_diagnostico','fecha_internacion','fecha_cui_intensivo','fecha_fallecimiento']]
casos.fecha_diagnostico.dropna(inplace=True)

In [ ]:
# Funcion para devolver base de fechas
def base(df,fecha_desde):
  ids = df[['residencia_provincia_id']].drop_duplicates()
  df = pd.DataFrame(columns=["fecha"])
  date = fecha_desde
  ayer = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
  fechas = pd.DataFrame(pd.date_range(date,ayer)).rename(columns={0:'fecha'})
  #today = pd.to_datetime('today').normalize() # fecha hasta
  final = pd.DataFrame(columns=['residencia_provincia_id','fecha'])
  for fecha in fechas['fecha']:
    base = ids
    base['fecha'] = fecha
    final = final.append(base, ignore_index=True)
  return final

In [ ]:
# CASOS
# Armar Cada Dataset a Joinear 
diag = casos.dropna(subset=['fecha_diagnostico'], how='all').groupby(['residencia_provincia_id','fecha_diagnostico'])['id_evento_caso'].count().reset_index()
diag.rename(columns={'id_evento_caso':'cuenta','fecha_diagnostico':'fecha'},inplace=True)
diag['fecha'] = pd.to_datetime(diag['fecha']) # convertir fecha a campot datetime
inter = casos.dropna(subset=['fecha_internacion'], how='all').groupby(['residencia_provincia_id','fecha_internacion'])['id_evento_caso'].count().reset_index()
inter.rename(columns={'id_evento_caso':'cuenta','fecha_internacion':'fecha'},inplace=True)
inter['fecha'] = pd.to_datetime(inter['fecha']) # convertir fecha a campot datetime
uti = casos.dropna(subset=['fecha_cui_intensivo'], how='all').groupby(['residencia_provincia_id','fecha_cui_intensivo'])['id_evento_caso'].count().reset_index()
uti.rename(columns={'id_evento_caso':'cuenta','fecha_cui_intensivo':'fecha'},inplace=True)
uti['fecha'] = pd.to_datetime(uti['fecha']) # convertir fecha a campot datetime
muertes = casos.dropna(subset=['fecha_fallecimiento'], how='all').groupby(['residencia_provincia_id','fecha_fallecimiento'])['id_evento_caso'].count().reset_index()
muertes.rename(columns={'id_evento_caso':'cuenta','fecha_fallecimiento':'fecha'},inplace=True)
muertes['fecha'] = pd.to_datetime(muertes['fecha']) # convertir fecha a campot datetime

In [ ]:
# CASOS
# Joinear el Dataset final
df_casos = pd.merge(base(casos,'2020-01-01'),diag,how='left',on=['residencia_provincia_id','fecha']) # partiendo del dataset Base
df_casos = pd.merge(df_casos,inter,how='left',on=['residencia_provincia_id','fecha'],suffixes=('_diag','_inter'))
df_casos = pd.merge(df_casos,uti,how='left',on=['residencia_provincia_id','fecha']).rename(columns={'cuenta':'cuenta_uti'})
df_casos = pd.merge(df_casos,muertes,how='left',on=['residencia_provincia_id','fecha']).rename(columns={'cuenta':'cuenta_muertes'}).fillna(0)

In [ ]:
# Importación Dataset Vacunas
# 
start_time = time.time()
vacunas = None
while vacunas is None:
  try:
    vacunas = pd.read_csv('https://sisa.msal.gov.ar/datos/descargas/covid-19/files/datos_nomivac_covid19.zip')
  except:
    pass
print("--- %s seconds ---" % (time.time() - start_time))

--- 325.86232900619507 seconds ---


In [ ]:
# Quedarnos con las columnas que vamos a usar y limpiar registros incompletos
vacunas = vacunas[['jurisdiccion_residencia_id','fecha_aplicacion','condicion_aplicacion','orden_dosis','sexo','vacuna']]
vacunas = vacunas[vacunas.fecha_aplicacion != 'S.I.']
vacunas.fecha_aplicacion.dropna(inplace=True)

In [ ]:
# Armar Cada Dataset a Joinear con datos de Vacunas
# Columnas de Condición de Aplicación:
# Salud -> condicion_aplicacion = 'Salud'
# Otros -> condicion_aplicacion = 'Otros'
# Estrategico -> condicion_aplicacion = 'Estrategico'
# Mayores_60 -> condicion_aplicacion = '60 o más años'
# Menores_60_SF -> '18 a 39 anos SIN Factores de Riesgo' o '40 a 49 anos SIN Factores de Riesgo' o '50 a 59 anos SIN Factores de Riesgo'
# Menores_60_CF -> '18 a 49 anos CON Factores de Riesgo'
#
# columnas tipo vacuna
# AstraZeneca -> vacuna = 'AstraZeneca'
# COVISHIELD  -> vacuna = 'COVISHIELD'
# Sinopharm   -> vacuna = 'Sinopharm'
# Sputnik     -> vacuna = 'Sputnik'
# Otra        -> vacuna not in ['AstraZeneca','COVISHIELD','Sinopharm','Sputnik'] (para identificar nuevas vacunas no tipificadas por ejemplo Moderna)
#
# Dosis:
# Primera_dosis -> orden_dosis = 1
# Segunda_dosis -> orden_dosis = 2
# Renombrar Columnas entre datasets
vacunas.rename(columns={'fecha_aplicacion':'fecha','jurisdiccion_residencia_id':'residencia_provincia_id'},inplace=True)
vacunas = vacunas.loc[vacunas.fecha != 'S.I.']
# Armar Cada Dataset a Joinear con datos de Vacunas
vac_x_cond = vacunas.pivot_table(index=['fecha','residencia_provincia_id'],columns='condicion_aplicacion',values='sexo',aggfunc='count').reset_index().fillna(0)
vac_x_cond['fecha'] = pd.to_datetime(vac_x_cond['fecha']) # convertir fecha a campo datetime
vac_x_vac = vacunas.pivot_table(index=['fecha','residencia_provincia_id'],columns='vacuna',values='sexo',aggfunc='count').reset_index().fillna(0)
vac_x_vac['fecha'] = pd.to_datetime(vac_x_vac['fecha']) # convertir fecha a campo datetime
vac_x_dosis = vacunas.pivot_table(index=['fecha','residencia_provincia_id'],columns='orden_dosis',values='sexo',aggfunc='count').reset_index().fillna(0)
vac_x_dosis['fecha'] = pd.to_datetime(vac_x_dosis['fecha']) # convertir fecha a campo datetime

In [ ]:
# Unir datasets de Vacunas
parcial = pd.merge(base(vacunas,'2020-01-01'),vac_x_cond,how='left',on=['residencia_provincia_id','fecha'])
#del vac_x_cond
parcial_2 = pd.merge(parcial,vac_x_vac,how='left',on=['residencia_provincia_id','fecha'])
#del parcial,vac_x_vac
df_vacunas = pd.merge(parcial_2,vac_x_dosis,how='left',on=['residencia_provincia_id','fecha'])

In [ ]:
#Unificar final de Casos y Vacunas
parcial = pd.merge(base(casos,'2020-01-01'),df_casos,how='left',on=['residencia_provincia_id','fecha'])
final = pd.merge(parcial,df_vacunas,how='left',on=['residencia_provincia_id','fecha'])

In [ ]:
# Importar población por provincia
provincias = pd.read_csv('https://raw.githubusercontent.com/alvarov0907/C2020/gh-pages/Poblacion_por_provincia.csv')
# Joinear al dataset
final_prov = pd.merge(final,provincias,how='left',on=['residencia_provincia_id'])

In [ ]:
# Descargar CSV
final_prov.to_csv('final_prov.csv',index=False)
files.download("final_prov.csv")

In [ ]:
# Data Provincias y Departamentos de Argentina para joinear
provincias = pd.read_csv('https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.7/download/provincias.csv')
departamentos = pd.read_csv('https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.8/download/departamentos.csv')